# Code-to-Code Comparison: Dinwoodie

### National Renewable Energy Laboratory

#### Rob Hammond

##### 31 October 2022

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import DINWOODIE, load_yaml

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base",
    "more_ctvs",
    "fewer_ctvs",
    "more_techs",
    "fewer_techs",
    "failure_50",
    "failure_200",
    "no_hlvs",
    "no_weather",
    "historic_weather",
    "manual_resets_only",
    "minor_repairs_only",
    "medium_repairs_only",
    "major_repairs_only",
    "major_replacements_only",
    "annual_service_only",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
    
}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(34)} | {(end - start) / 60:.2f} m")
    
    # Gather the results of interest
    years = sim.metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability_time = sim.metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
    availability_production = sim.metrics.production_based_availability(frequency="project", by="windfarm").values[0][0]
    cf_net = sim.metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
    cf_gross = sim.metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
    power_production = sim.metrics.power_production(frequency="project", by_turbine=False).values[0][0]
    completion_rate = sim.metrics.task_completion_rate(which="both", frequency="project").values[0][0]
    parts = sim.metrics.events[["materials_cost"]].sum().sum()
    techs = sim.metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    total = sim.metrics.events[["total_cost"]].sum().sum()
    
    equipment = sim.metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]].sum().sum()
    fsv = equipment[[el for el in equipment.columns if "Field Support Vessel" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    
    utilization = sim.metrics.service_equipment_utilization(frequency="project")
    hlv_ur = utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]].mean().mean()
    fsv_ur = utilization[[el for el in utilization.columns if "Field Support Vessel" in el]].mean().mean()
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)
    sim.env.cleanup_log_files(log_only=True)

                    dinwoodie_base | 1.14 m
               dinwoodie_more_ctvs | 1.04 m
              dinwoodie_fewer_ctvs | 1.06 m
              dinwoodie_more_techs | 1.23 m
             dinwoodie_fewer_techs | 1.23 m
              dinwoodie_failure_50 | 1.05 m
             dinwoodie_failure_200 | 1.62 m
                 dinwoodie_no_hlvs | 1.23 m
              dinwoodie_no_weather | 1.07 m
        dinwoodie_historic_weather | 0.96 m
      dinwoodie_manual_resets_only | 1.16 m
      dinwoodie_minor_repairs_only | 0.89 m
     dinwoodie_medium_repairs_only | 0.74 m
      dinwoodie_major_repairs_only | 0.73 m
 dinwoodie_major_replacements_only | 0.64 m
     dinwoodie_annual_service_only | 0.89 m


In [5]:
# Save the results
with open(DINWOODIE / "outputs" / "results_dict_v0.5.2.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(DINWOODIE / "outputs" / "results_data_v0.5.2.csv", index_label="result")

In [6]:
results_df

,base,more_ctvs,fewer_ctvs,more_techs,fewer_techs,failure_50,failure_200,no_hlvs,no_weather,historic_weather,manual_resets_only,minor_repairs_only,medium_repairs_only,major_repairs_only,major_replacements_only,annual_service_only
availability - time based,0.97,0.96,0.96,0.96,0.96,0.97,0.96,0.97,0.97,0.96,1.00,1.00,1.00,1.00,0.98,0.98
availability - production based,0.97,0.96,0.96,0.96,0.96,0.97,0.96,0.98,0.00,0.96,1.00,1.00,1.00,1.00,0.98,0.99
capacity factor - net,0.46,0.46,0.46,0.46,0.46,0.46,0.45,0.46,0.00,0.46,0.47,0.47,0.47,0.47,0.47,0.47
capacity factor - gross,0.48,0.47,0.47,0.47,0.47,0.47,0.47,0.47,0.00,0.48,0.47,0.47,0.47,0.47,0.47,0.47
power production,"9,689,447,404.00","9,202,636,326.50","9,212,960,546.50","9,203,658,931.50","9,195,640,658.00","9,244,468,915.00","9,149,370,264.00","9,338,389,653.50",0.00,"7,757,642,377.00","9,533,883,785.50","9,532,155,936.00","9,557,712,929.50","9,572,001,703.50","9,391,027,692.50","9,433,592,991.00"
task completion rate,1.00,1.00,0.99,1.00,0.99,1.00,1.00,1.00,1.00,0.99,1.00,1.00,1.00,1.00,0.93,0.90
annual direct O&M cost,13.39,16.04,13.34,13.09,11.70,10.49,14.73,5.40,11.56,12.30,3.45,3.72,3.91,4.73,11.82,4.50
annual vessel cost,8.86,9.44,9.10,8.05,8.34,6.65,9.81,1.89,7.07,8.01,1.91,1.80,1.78,1.97,7.81,1.76
ctv cost,1.92,3.18,1.78,1.81,1.77,1.84,1.91,1.89,1.89,1.75,1.91,1.80,1.78,1.80,1.66,1.76
fsv cost,0.14,0.09,0.07,0.07,0.13,0.05,0.20,0.00,0.11,0.05,0.00,0.00,0.00,0.17,0.00,0.00
